<a href="https://colab.research.google.com/github/trentfridey/qosf-screening/blob/master/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pennylane
!pip install pennylane-cirq

     |████████████████████████████████| 317kB 2.7MB/s 
     |████████████████████████████████| 1.4MB 4.0MB/s 


# Task 2

In this notebook, we explore implementing a circuit operating on two qubits initially in the state $|00\rangle$ to return the states $|01\rangle$ and $|10\rangle$ with equal probability. The circuit is a variational circuit composed of an $R_x$ gate, an $R_y$ gate, and a $CNOT$ gate. It is subject to bitflip noise on both qubits. We see how it performs for a limited number of measurements

## Analysis

In order to train the parameters of the circuit (i.e. the angles of the $R_x$ and $R_y$ gates), we define a cost function:

$$
C(|\psi(\theta)\rangle) = 2-\langle \psi(\theta)| U | \psi(\theta)\rangle
$$

Where $U$ is the unitary operator:

$$
U = \left(\begin{matrix} 1 & 0 & 0 & 0 \\
  0 & 0 & 2 & 0 \\
  0 & 2 & 0 & 0 \\
  0 & 0 & 0 & 1
  \end{matrix}\right)
$$

By inspection, this matrix has the eigenvector $\frac{1}{\sqrt2}\left(|01\rangle + |10\rangle\right)$ with the highest eigenvalue $2$. Hence, optimizing for the expectation value to equal its highest eigenvalue will lead the circuit to prepare the desired state

The ansatz for the circuit is $CNOT*R_y(\theta_y)\otimes R_x(\theta_x)$

In [1]:
import pennylane as qml
import numpy as np
from pennylane_cirq import ops

In [39]:
# Definitions

observable = qml.Hermitian(np.array([[1,0,0,0],[0,0,2,0],[0,2,0,0],[0,0,0,1]]), wires=[0,1])

def compute_circ_params(n_shots):
  
  # Device setup

  dev = qml.device('cirq.mixedsimulator', wires=2, shots=n_shots, analytic=False)

  @qml.qnode(dev)
  def ansatz(thetas):
    qml.RY(thetas[0], wires=0)
    qml.RX(thetas[1], wires=1)
    qml.CNOT(wires=[0,1])

    # Here we add in the BitFlip operator for modelling noise
    ops.BitFlip(0.5, wires=0)
    ops.BitFlip(0.5, wires=1)
    return qml.expval(observable)

  def cost_fn(thetas):
    return 2 - ansatz(thetas)

  # Optimization

  steps = 200

  init_params = np.random.normal(0, np.pi, 2)

  params = init_params

  opt = qml.GradientDescentOptimizer(stepsize=0.4)

  for i in range(steps):
    params = opt.step(cost_fn, params)

  print("Optimized: angles: {}, {} | cost: {}".format(params[0], params[1], cost_fn(params)))
  return params



In [40]:
optimized = []
for n in [1,10,100,1000,1000]:
  optimized += [compute_circ_params(n)]


Optimized: angles: 1.7899020447955365, 0.11924838313987218 | cost: 1.0
Optimized: angles: 1.5473472899629015, -0.5908439593496232 | cost: 0.5
Optimized: angles: -4.69396942362689, -5.030745485015755 | cost: 0.51
Optimized: angles: -4.708156126380034, -1.5692346171797202 | cost: 0.506
Optimized: angles: -4.726643779709236, 4.756117147357814 | cost: 0.4810000000000001


## Conclusion

By allowing more samples in our trials, we can see some indication that the minimum cost function is improving. This may also be balanced with the probability of getting a bitflip during each trial, so more analysis regarding where the balance lies could be beneficial